In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from synthlearners.utils import prepare_panel
from synthlearners.crossvalidation import PanelCrossValidator

In [ ]:
data = prepare_panel(
    adh_data,
    unit_col="country",
    time_col="year",
    outcome_col="gdp",
    treatment_col="treat",
)

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(10, 6), sharey=True)
ax[0].matshow(data["Y"], aspect="auto", cmap="viridis")
ax[1].matshow(data["W"], aspect="auto", cmap="viridis")
f.tight_layout()
ax[0].set_title("Outcome matrix")
ax[1].set_title("Treatment matrix")

In [ ]:
n_split_cnt = 4
cv = PanelCrossValidator(n_splits=n_split_cnt)

In [ ]:
def plot_masks(m):
    f, ax = plt.subplots(len(m), 2, figsize=(10, 10), sharey=True)

    for n in range(len(m)):
        ax[n,0].matshow(m[n][0], aspect="auto", cmap="viridis")
        ax[n,1].matshow(m[n][1], aspect="auto", cmap="viridis")

    f.tight_layout()
    ax[0,0].set_title("Train matrix")
    ax[0,1].set_title("Test matrix")

In [ ]:
horizontal_masks = cv.horizontal_split(data["Y"])
plot_masks(horizontal_masks)

In [ ]:
vertical_masks = cv.vertical_split(data["Y"])
plot_masks(vertical_masks)

In [ ]:
box_masks = cv.box_split(data["Y"])
plot_masks(box_masks)

In [ ]:
random_masks = cv.random_split(data["Y"])
plot_masks(random_masks)